<a href="https://colab.research.google.com/github/thanhnhan311201/CS114.L11.KHCL/blob/master/Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code : Nguyễn Thành Nghĩa
# Mô tả bài toán: Phân loại tiêu đề mang tính chất trào phúng. 
- Dữ liệu đầu vào: là các dòng tiêu đề có tính chất trào phúng và những tiêu đề không có tính chất trào phúng.
- Yêu cầu: cho một bộ những tiêu đề chưa được phân loại, hãy xác định xem chúng thuộc loại có tính chất trào phúng hay không.

# Ý tưởng
- Lọc ra những từ có tác động đáng kể đến dataset 
- Đưa các dòng tiêu đề của dữ liệu đầu vào về dạng túi từ (bag-of-words)
- Dùng mô hình Multinominal Naive Bayes với dữ liệu vừa xử lý để dự đoán kết quả

# Link dataset: https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection
@article{misra2019sarcasm,
  title={Sarcasm Detection using Hybrid Neural Network},
  author={Misra, Rishabh and Arora, Prahal},
  journal={arXiv preprint arXiv:1908.07414},
  year={2019}
}


In [ ]:
# Clone từ github để lấy dữ liệu
!git clone https://github.com/thanhnhan311201/CS114.L11.KHCL.git

In [ ]:
!pip install stop-words

In [ ]:
# Thêm một số thư viện cần thiết
from nltk.corpus import stopwords
import nltk
import json
import numpy as np
import pandas as pd
from collections import deque 


In [ ]:
#Load dataset
file_ = pd.read_json("/content/CS114.L11.KHCL/LogisticsRegression/Sarcasm_Project/Sarcasm_Headlines_Dataset.json",lines=True)
file_.head()
del file_['article_link']
# Training data // dùng 20000 dòng của dataset làm training data
data = file_.to_numpy()[:20000,:]
# Testing data // khoảng 6000 dòng còn lại làm testing data
data2 = file_.to_numpy()[20000:,:]


In [ ]:
# Lọc lại văn bản, chỉ giữ lại chữ và số
text = ''
for i in range(len(data)):
  text += data[i,0]
text = deque(text)
while i < len(text):
  if not((text[i] >='a' and text[i] <='z') or (text[i] >='0' and text[i] <='9') or (text[i] == ' ')):
    text.remove(text[i])
    i-=1
  i+=1
text = ''.join(text)

In [ ]:
# thống kê số lượng từ xuất hiện trong toàn bộ dataset
text = deque(text.split())
dic = {}
for i in range(len(text)):
  dic[text[i]] = text.count(text[i])

In [ ]:
# download bộ những từ không có ý nghĩa nhiều trong tiếng anh
nltk.download('stopwords')
stop = set(stopwords.words('english'))

In [ ]:

# Lọc bỏ ra những từ xuất hiện ít trong dataset có thể gây nhiễu và chia thành dic thành 2 list tiện cho xử lý
# số num có tính tương đối
num = 10
val = list(i for i in dic.values() if i > num)
key = list(i for i in dic.keys() if dic[i] > num)

# Tính độ lệch chuẩn tần suất xuất hiện của các từ
std = np.std(val)
mean = np.mean(val)

count = 0
filtered_words = []
# Lọc ra những từ có ảnh hưởng tưởng đối đến dataset
for i in range(len(val)):
  if key[i] not in stop and val[i] >= abs(mean-std) and val[i] <= abs(mean+std):
    filtered_words.append(key[i])
    count +=1

In [ ]:
# hàm Chuyển 1 câu sang vectors từ (bag words)
def Sentences2Vector(s,w):
  v = []
  for i in range(len(w)):
    v.append(s.count(w[i]))
  return v
# chuyển training data sang dạng vector
vectors = []
for i in range(len(data)):
  vectors.append(Sentences2Vector(data[i,0],filtered_words))
vectors = np.array(vectors)


v_sarcasm = []
v_none_sarcasm = []
# Phân loại traning data ra sarcasm và none sarcasm
for i in range(len(vectors)):
  label = data[i,-1]
  t = vectors[i,:] 
  if label == 1:
    v_sarcasm.append(t)
  if label == 0:
    v_none_sarcasm.append(t)

# chuyển sang numpy
v_sarcasm = np.array(v_sarcasm) 
v_none_sarcasm = np.array(v_none_sarcasm)


# Sử dụng Model Multinomial Naive Bayes để dự đoán


In [ ]:
# Hàm Smooth
def Smooth(v):
  if 0 in v:
    v = v + 1
  v = v/sum(v)
  return v

In [ ]:
# Tính tổng các vector của 2 labels
sum_v_sarcasm = np.sum(v_sarcasm,axis=0)
sum_v_none_sarcasm = np.sum(v_none_sarcasm,axis=0)
# Smooth lại kết quả
smooth_v_sarcasm = Smooth(sum_v_sarcasm)
smooth_v_none_sarcasm = Smooth(sum_v_none_sarcasm)


In [ ]:
# Chuyển Testing data sang dạng vector
v_test = []
for i in range(len(data2)):
  v_test.append(Sentences2Vector(data2[i,0],filtered_words))
v_test = np.array(v_test)

In [ ]:
S = len(v_sarcasm)
N = len(v_none_sarcasm)
total = S + N
result = []
count = 0
error = 0
# Tính toán và dự đoán kết quả của từng testcase
for i in v_test:
  PS = S/total 
  PN = N/total
  RS = PS * np.prod(pow(smooth_v_sarcasm,i)) # Xác suất tin sarcasm
  RN = PN * np.prod(pow(smooth_v_none_sarcasm,i)) # Xác suất tin none sarcasm
  if RS >= RN:
    result.append(1)
  else:
    result.append(0)
  # Kết quả dự đoán khác kết quả thực tế tăng error lên 1
  if result[count] != data[count,1]:
    error += 1
  count += 1

# In ra độ sai lệch
print("Error = {}%".format(round(error*100/len(data2),3)))




Error = 45.074%
